In [143]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
from keras import layers
import optuna
from keras.models import Sequential,load_model,clone_model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.losses import MeanSquaredError,MeanAbsoluteError
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,root_mean_squared_error
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import joblib
from scikeras.wrappers import KerasRegressor
from sklearn.base import BaseEstimator, RegressorMixin

In [144]:
data=pd.read_csv('processed_data\preprocessed_data.csv')
data.head()

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\aishw\AppData\Local\Temp\ipykernel_39104\2185253439.py:1: SyntaxWarning: invalid escape sequence '\p'
  data=pd.read_csv('processed_data\preprocessed_data.csv')


,year,price,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Automatic,transmission_Semi-Auto,transmission_Other,fuelType_Petrol,fuelType_Diesel,fuelType_Hybrid,fuelType_Other,fuelType_Electric,model_encoded
0,2017,12500,15735,150.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14327.750557
1,2016,16500,36203,20.0,64.2,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22695.385027
2,2016,11000,29946,30.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14327.750557
3,2017,16800,25952,145.0,67.3,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20255.450398
4,2019,17300,1998,145.0,49.6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17408.522032


In [51]:
data.shape

(94908, 16)

In [145]:
from scipy.stats import zscore
columns_to_check=['mileage','tax','mpg','engineSize']
z_scores=np.abs(zscore(data[columns_to_check]))
data_clean=data[(z_scores < 3).all(axis=1)]
print(f"Before: {data.shape}\nAfter: {data_clean.shape}")

Before: (94908, 16)
After: (91788, 16)


In [153]:
data_clean.head()

,year,price,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Automatic,transmission_Semi-Auto,transmission_Other,fuelType_Petrol,fuelType_Diesel,fuelType_Hybrid,fuelType_Other,fuelType_Electric,model_encoded
0,2017,12500,15735,150.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14327.750557
1,2016,16500,36203,20.0,64.2,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22695.385027
2,2016,11000,29946,30.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14327.750557
3,2017,16800,25952,145.0,67.3,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20255.450398
4,2019,17300,1998,145.0,49.6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17408.522032


In [146]:
x=data_clean.drop('price',axis=1).values
y=data_clean['price'].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
model=Sequential([
    layers.Dense(units=128),
    layers.Dense(units=64),
    layers.Dense(units=32),
    layers.Dense(units=1,activation='linear')
])
model.compile(loss='mean_absolute_error',
              optimizer=Adam(),
              metrics=['mae'])
history=model.fit(x_train,y_train,
          epochs=10,
          validation_data=(x_test,y_test),
          batch_size=32)

Epoch 1/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 30s 6ms/step - loss: 3625.4688 - mae: 3625.4688 - val_loss: 3328.4468 - val_mae: 3328.4468
Epoch 2/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3309.7932 - mae: 3309.7932 - val_loss: 3330.3872 - val_mae: 3330.3872
Epoch 3/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3305.8457 - mae: 3305.8457 - val_loss: 3241.4133 - val_mae: 3241.4133
Epoch 4/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3305.6082 - mae: 3305.6082 - val_loss: 3241.7087 - val_mae: 3241.7087
Epoch 5/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3279.0698 - mae: 3279.0698 - val_loss: 3240.9038 - val_mae: 3240.9038
Epoch 6/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3291.0398 - mae: 3291.0398 - val_loss: 3228.3149 - val_mae: 3228.3149
Epoch 7/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3266.4158 - mae: 3266.4158 - val_loss: 3263.6167 - val_mae: 3263.6167
Epoch 8/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 3253.4

In [7]:
model.evaluate(x_test,y_test)

594/594 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 3240.7910 - mae: 3240.7910


[3213.31201171875, 3213.31201171875]

In [9]:
y_pred=model.predict(x_test)
y_pred

594/594 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


array([[12771.626],
       [12855.751],
       [27114.92 ],
       ...,
       [11008.469],
       [10043.884],
       [11016.74 ]], dtype=float32)

In [15]:
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')


Mean Squared Error: 27150362.0
Mean Absolute Error: 3213.3134765625
Root Mean Squared Error: 5210.60107421875
r2 score: 0.542769193649292


In [ ]:
model1=Sequential([
    layers.Dense(units=128,activation='relu'),
    layers.Dense(units=64,activation='relu'),
    layers.Dense(units=32,activation='relu'),
    layers.Dense(units=1,activation='linear')
])
model1.compile(loss='mean_absolute_error',
              optimizer=Adam(),
              metrics=['mae'])
history1=model1.fit(x_train,y_train,
          epochs=10,
          validation_data=(x_test,y_test),
          batch_size=32)

Epoch 1/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 3662.2583 - mae: 3662.2583 - val_loss: 2828.8464 - val_mae: 2828.8464
Epoch 2/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 2823.3501 - mae: 2823.3501 - val_loss: 2776.9194 - val_mae: 2776.9194
Epoch 3/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 2832.6782 - mae: 2832.6782 - val_loss: 2825.4243 - val_mae: 2825.4243
Epoch 4/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 2823.7317 - mae: 2823.7317 - val_loss: 2819.8743 - val_mae: 2819.8743
Epoch 5/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - loss: 2788.6389 - mae: 2788.6389 - val_loss: 2759.3242 - val_mae: 2759.3242
Epoch 6/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 2766.1675 - mae: 2766.1675 - val_loss: 2731.1016 - val_mae: 2731.1016
Epoch 7/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 2756.2385 - mae: 2756.2385 - val_loss: 2788.3271 - val_mae: 2788.3271
Epoch 8/10
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - loss: 2746.0

In [17]:
model1.evaluate(x_test,y_test)

594/594 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 2766.5806 - mae: 2766.5806


[2749.8662109375, 2749.8662109375]

In [18]:
y_pred=model1.predict(x_test)
y_pred

594/594 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


array([[12872.07 ],
       [13406.5  ],
       [30943.977],
       ...,
       [10811.278],
       [10168.525],
       [ 9981.85 ]], dtype=float32)

In [19]:
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

Mean Squared Error: 19525706.0
Mean Absolute Error: 2749.866455078125
Root Mean Squared Error: 4418.7900390625
r2 score: 0.7486937642097473


## Training for longer and adding dropouts and kernel regularizers

In [9]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [47]:
lr_schedule = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    verbose=1,
    min_lr=1e-7
)


In [39]:
model2 = Sequential([
    layers.Dense(128, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(64, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(32, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(1, activation='linear')
])

model2.compile(loss='mean_absolute_error',
              optimizer=Adam(learning_rate=0.001),
              metrics=['mae'])
history2=model2.fit(x_train,y_train,
          epochs=50,
          validation_data=(x_test,y_test),
          batch_size=32,
          callbacks=[early_stop,lr_schedule])

Epoch 1/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - loss: 17815.6719 - mae: 17815.5293 - val_loss: 17253.8926 - val_mae: 17253.7031 - learning_rate: 0.0010
Epoch 2/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 17206.9141 - mae: 17206.6895 - val_loss: 16224.7334 - val_mae: 16224.4346 - learning_rate: 0.0010
Epoch 3/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 16081.5957 - mae: 16081.2783 - val_loss: 14715.5850 - val_mae: 14715.2002 - learning_rate: 0.0010
Epoch 4/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 14336.3926 - mae: 14335.9834 - val_loss: 12727.1133 - val_mae: 12726.6309 - learning_rate: 0.0010
Epoch 5/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 12225.1729 - mae: 12224.6709 - val_loss: 10350.1934 - val_mae: 10349.6445 - learning_rate: 0.0010
Epoch 6/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - loss: 9886.4639 - mae: 9885.8867 - val_loss: 7672.9658 - val_mae: 7672.3120 - learning_rate: 0.0010
Epoch 7/50
2373/2373 ━━━━━━━━━━━━━━━━━

In [40]:
model2.evaluate(x_test,y_test)

594/594 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 2704.9915 - mae: 2700.8386


[2695.483642578125, 2691.329345703125]

In [41]:
y_pred=model2.predict(x_test)
y_pred

594/594 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step


array([[11602.002],
       [12590.745],
       [28329.57 ],
       ...,
       [ 9980.295],
       [ 9503.99 ],
       [ 9673.891]], dtype=float32)

In [42]:
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

Mean Squared Error: 20417952.0
Mean Absolute Error: 2691.330078125
Root Mean Squared Error: 4518.62255859375
r2 score: 0.6573730707168579


In [11]:
model3 = Sequential([
    layers.Dense(128, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(64, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(32, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(1, activation='linear')
])

model3.compile(loss=tf.keras.losses.Huber(),
              optimizer=Adam(learning_rate=0.001),
              metrics=['mae'])
history3=model3.fit(x_train,y_train,
          epochs=50,
          validation_data=(x_test,y_test),
          batch_size=32,
          callbacks=[early_stop,lr_schedule])

Epoch 1/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 39s 8ms/step - loss: 17796.6582 - mae: 17797.0332 - val_loss: 17265.7285 - val_mae: 17266.0352 - learning_rate: 0.0010
Epoch 2/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - loss: 17211.2871 - mae: 17211.5781 - val_loss: 16177.8896 - val_mae: 16178.1035 - learning_rate: 0.0010
Epoch 3/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - loss: 15881.5537 - mae: 15881.7168 - val_loss: 14602.2969 - val_mae: 14602.3945 - learning_rate: 0.0010
Epoch 4/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - loss: 14212.7178 - mae: 14212.7861 - val_loss: 12664.9795 - val_mae: 12664.9932 - learning_rate: 0.0010
Epoch 5/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 12132.1094 - mae: 12132.0967 - val_loss: 10085.5322 - val_mae: 10085.4395 - learning_rate: 0.0010
Epoch 6/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 9679.9033 - mae: 9679.7832 - val_loss: 7473.4497 - val_mae: 7473.2197 - learning_rate: 0.0010
Epoch 7/50
2373/2373 ━━━━━━━━━━━━━━━━━

In [12]:
model3.evaluate(x_test,y_test)

594/594 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 2646.1292 - mae: 2643.1746


[2629.81982421875, 2626.86669921875]

In [13]:
y_pred=model3.predict(x_test)
y_pred

594/594 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


array([[12583.499],
       [13148.214],
       [30964.246],
       ...,
       [ 9988.222],
       [ 9733.652],
       [ 9927.144]], dtype=float32)

In [14]:
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

Mean Squared Error: 18098778.0
Mean Absolute Error: 2626.865966796875
Root Mean Squared Error: 4254.265625
r2 score: 0.7195514440536499


In [31]:
model3.save('models/dl_model.h5')

In [47]:
model4 = Sequential([

    layers.Dense(64, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(32, kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(1, activation='linear')
])

model4.compile(loss=tf.keras.losses.Huber(),
              optimizer=Adam(learning_rate=0.001),
              metrics=['mae'])
history4=model4.fit(x_train,y_train,
          epochs=50,
          validation_data=(x_test,y_test),
          batch_size=32,
          callbacks=[early_stop,lr_schedule])

Epoch 1/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - loss: 17819.6504 - mae: 17820.1074 - val_loss: 17238.5254 - val_mae: 17238.9277 - learning_rate: 0.0010
Epoch 2/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - loss: 17163.8008 - mae: 17164.1875 - val_loss: 16206.0693 - val_mae: 16206.4170 - learning_rate: 0.0010
Epoch 3/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 15941.2354 - mae: 15941.5605 - val_loss: 14467.7539 - val_mae: 14468.0273 - learning_rate: 0.0010
Epoch 4/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 14224.9600 - mae: 14225.2148 - val_loss: 12487.6445 - val_mae: 12487.8623 - learning_rate: 0.0010
Epoch 5/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - loss: 12140.6006 - mae: 12140.8076 - val_loss: 10035.1064 - val_mae: 10035.2529 - learning_rate: 0.0010
Epoch 6/50
2373/2373 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 9652.6113 - mae: 9652.7412 - val_loss: 7208.6616 - val_mae: 7208.7104 - learning_rate: 0.0010
Epoch 7/50
2373/2373 ━━━━━━━━━━━━━━━━━

In [48]:
model4.evaluate(x_test,y_test)

594/594 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 2575.5537 - mae: 2573.6411


[2563.959228515625, 2562.0478515625]

In [49]:
y_pred=model4.predict(x_test)
y_pred

594/594 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


array([[12332.442],
       [14296.904],
       [28036.912],
       ...,
       [ 9011.653],
       [ 8557.162],
       [ 9606.884]], dtype=float32)

In [50]:
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

Mean Squared Error: 20564910.0
Mean Absolute Error: 2562.04833984375
Root Mean Squared Error: 4534.85498046875
r2 score: 0.6690679788589478


## Implementing Stacking Regressor

In [5]:
xgb_model=joblib.load('models/best_xgb_model.pkl')
y_pred=xgb_model.predict(x_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

Mean Squared Error: 3069136.25
Mean Absolute Error: 1108.296142578125
Root Mean Squared Error: 1751.89501953125
r2 score: 0.9673832654953003


In [18]:
dl_model=joblib.load('models/dl_model.h5')
y_pred=dl_model.predict(x_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

594/594 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Mean Squared Error: 18098778.0
Mean Absolute Error: 2626.865966796875
Root Mean Squared Error: 4254.265625
r2 score: 0.7195514440536499


In [30]:
import os
print(os.path.exists('models/dl_model.h5'))  # Should return True


True


In [35]:
dl_model=load_model('models/dl_model.h5')
dl_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,315 (52.02 KB)

 Trainable params: 12,865 (50.25 KB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 2 (12.00 B)

In [41]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
import numpy as np

def keras_predict(X):
    preds = dl_model.predict(np.array(X), verbose=0)
    return np.squeeze(preds).reshape(-1, 1)

dl_pipeline = make_pipeline(
    FunctionTransformer(func=keras_predict, validate=False)
)

In [121]:
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.models import load_model
import numpy as np


from sklearn.base import BaseEstimator, RegressorMixin

class MyKerasRegressor(BaseEstimator, RegressorMixin):
    _estimator_type = "regressor"
    def __init__(self, input_dim, epochs=10, batch_size=32, verbose=0):
        self.input_dim = input_dim
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose


    def _build_model(self):
        model = Sequential()
        model.add(layers.Dense(64, activation='relu', input_shape=(self.input_dim,)))
        model.add(layers.Dense(32, activation='relu'))
        model.add(layers.Dense(1))
        model.compile(optimizer=Adam(), loss='mse')
        return model

    def fit(self, X, y):
        self.model_ = self._build_model()
        self.model_.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self

    def predict(self, X):
        return self.model_.predict(X).flatten()  # flatten to shape (n_samples,)

    def get_params(self, deep=True):
        return {
            'input_dim': self.input_dim,
            'epochs': self.epochs,
            'batch_size': self.batch_size,
            'verbose': self.verbose
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self



dl_model = MyKerasRegressor(input_dim=x_train.shape[1], epochs=10, batch_size=32)


In [48]:
lr_schedule = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    verbose=1,
    min_lr=1e-7
)

In [126]:
#from scikeras.wrappers import KerasRegressor
from scikeras.wrappers import KerasRegressor


def build_model():
    model = Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=x_train.shape[1]))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer=Adam(), loss='mean_absolute_error',metrics=['mae'])
    return model


In [137]:
params=xgb_model.get_params()
params

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.6685732714024194,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'feature_weights': None,
 'gamma': 4.873705258433285,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.09783300643865717,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 16,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 117,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': 5.045910323808419,
 'reg_lambda': 7.572563029928501,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.9713652590548958,
 'tree_method': None,
 'va

In [138]:
new_xgb_model=XGBRegressor(**params)

In [166]:
params

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.6685732714024194,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'feature_weights': None,
 'gamma': 4.873705258433285,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.09783300643865717,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 16,
 'max_leaves': None,
 'min_child_weight': None,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 117,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': 5.045910323808419,
 'reg_lambda': 7.572563029928501,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.9713652590548958,
 'tree_method': None,
 'validate_parameters

In [161]:
params.pop('missing')

nan

In [178]:
new_xgb_model = XGBRegressor(**params)


In [176]:
x_train=x_train.astype(np.float64)
x_test=x_test.astype(np.float64)
y_train = y_train.astype(np.float64)
y_test=y_test.astype(np.float64)

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Try with only Lasso
base_model = [('lasso', Lasso(alpha=0.1))]
meta_model = Ridge()
stacking_model = StackingRegressor(estimators=base_model, final_estimator=meta_model)
stacking_model.fit(x_train, y_train)  # ✅ Should work

# Try with only Random Forest
base_model = [('rf', RandomForestRegressor())]
stacking_model = StackingRegressor(estimators=base_model, final_estimator=meta_model)
stacking_model.fit(x_train, y_train)  # ✅ Should work

# Try with only XGBoost
base_model = [('xgb', new_xgb_model)]
stacking_model = StackingRegressor(estimators=base_model, final_estimator=meta_model)
stacking_model.fit(x_train, y_train)  # 🔥 Likely to throw the same error


In [175]:
import numpy as np

# If x_train is a NumPy array
contains_dict = any(isinstance(x, dict) for x in np.ravel(x_train))
print("❌ Contains dicts?" if contains_dict else "✅ No dicts found")


✅ No dicts found


In [180]:
print(x_train.dtype)


float64


In [ ]:
"""dl_model=build_model()
dl_model.fit(x_train,y_train,
             epochs=50,
             validation_data=(x_test,y_test),
             batch_size=32)"""
from sklearn.model_selection import KFold
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor

cv = KFold(n_splits=5)

base_model = [
    ('lasso', Lasso(alpha=0.1)),
    ('rf', RandomForestRegressor(random_state=42)),
    ('xbg',new_xgb_model)
]

meta_model = Ridge(alpha=1.0)

stacking_model = StackingRegressor(
    estimators=base_model,
    final_estimator=meta_model,
    passthrough=True,
    cv=cv,
    n_jobs=1
)

stacking_model.fit(x_train, y_train)


d:\Code files\Used Car Price Prediction\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.221e+08, tolerance: 6.146e+08
  model = cd_fast.enet_coordinate_descent(
d:\Code files\Used Car Price Prediction\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.071e+11, tolerance: 4.894e+08
  model = cd_fast.enet_coordinate_descent(
d:\Code files\Used Car Price Prediction\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

StackingRegressor(cv=KFold(n_splits=5, random_state=RandomState(MT19937) at 0x25E4469BB40,
   shuffle=False),
                  estimators=[('lasso', Lasso(alpha=0.1)),
                              ('rf', RandomForestRegressor(random_state=42))],
                  final_estimator=Ridge(), n_jobs=1, passthrough=True)

In [160]:
y_pred = stacking_model.predict(x_test)
print(f'Mean Squared Error: {mean_squared_error(y_test1, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test1, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test1,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test1)}')

Mean Squared Error: 2780333.2785434676
Mean Absolute Error: 1071.8462475141755
Root Mean Squared Error: 1667.4331406516628
r2 score: 0.9650870120215769


In [130]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
import numpy as np

#lasso = Lasso(alpha=0.1).fit(x_train, y_train)
rf = RandomForestRegressor(random_state=42).fit(x_train, y_train)
xgb_model = joblib.load("models/best_xgb_model.pkl")
dl_model = load_model("models/dl_model.h5")

#train_lasso_pred = lasso.predict(x_train)
train_rf_pred = rf.predict(x_train)
train_xgb_pred = xgb_model.predict(x_train)
train_dl_pred = np.squeeze(dl_model.predict(x_train, verbose=0))

stacked_train = np.column_stack((train_rf_pred, train_xgb_pred, train_dl_pred))


In [131]:
meta_model = Ridge(alpha=1.0)
meta_model.fit(stacked_train, y_train)


Ridge()

In [132]:
#test_lasso_pred = lasso.predict(x_test)
test_rf_pred = rf.predict(x_test)
test_xgb_pred = xgb_model.predict(x_test)
test_dl_pred = np.squeeze(dl_model.predict(x_test, verbose=0))

stacked_test = np.column_stack((test_rf_pred, test_xgb_pred, test_dl_pred))


In [133]:
y_pred = meta_model.predict(stacked_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

Mean Squared Error: 3694895.1241122372
Mean Absolute Error: 1225.6688591914722
Root Mean Squared Error: 1922.2109988532054
r2 score: 0.9618280375422781


In [134]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
stacked_train_scaled = scaler.fit_transform(stacked_train)
stacked_test_scaled = scaler.transform(stacked_test)
meta_model.fit(stacked_train_scaled, y_train)


Ridge()

In [135]:
y_pred = meta_model.predict(stacked_test_scaled)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test,y_pred)}')
print(f'r2 score: {r2_score(y_pred,y_test)}')

Mean Squared Error: 3689902.5629029116
Mean Absolute Error: 1224.8224871224963
Root Mean Squared Error: 1920.911909199095
r2 score: 0.9618770469499374
